In [ ]:
import pytest
import simuOpt
simuOpt.setOptions(alleleType='short', quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
np.set_printoptions(suppress=True, precision=3)

In [ ]:
pf_map = shelve.open('pf_map')
misc_gmap = shelve.open('misc_gmap')
uniparams = shelve.open('uniparams')

locus_names = uniparams['locus_names']
pos_column = uniparams['pos_column']
allele_names = uniparams['allele_names']
snp_to_integer = uniparams['snp_to_integer']
integer_to_snp = uniparams['integer_to_snp']

alleles = misc_gmap['alleles']
chr_cM_positions = misc_gmap['chr_cM_positions']
cM_positions = misc_gmap['cM_positions']
integral_valued_loci = misc_gmap['integral_valued_loci']
relative_integral_valued_loci = misc_gmap['relative_integral_valued_loci']
recombination_rates = misc_gmap['recombination_rates']

In [ ]:
uniparams['founders'] = [[1, 2], [3, 4], [5, 6], [7, 8]]

In [ ]:
uniparams['operating_population_size']

In [ ]:
offspring_per_founder = 2000/8

In [ ]:
nam = sim.loadPopulation(uniparams['prefounder_file_name'])
sim.tagID(nam, reset=True)
nam.setSubPopName('maize_nam_prefounders', 0)

"""
selection_statistics = {
    'aggregate': {},
    'selected': {},
    'non-selected': {}
}
"""
ind_names_for_gwas = {i: {} for i in range(uniparams[
    'number_of_replicates'])}
uniparams['meta_pop_sample_sizes'] = {i: 100 for i in
                                      range(0, uniparams['generations_of_selection'] + 1, 2)
                                      }

s = simulate.Truncation(uniparams['generations_of_selection'],
                       uniparams['generations_of_random_mating'],
                       uniparams['operating_population_size'],
                        uniparams[
                            'proportion_of_individuals_saved'],
                       uniparams['overshoot_as_proportion'],
                   uniparams['individuals_per_breeding_subpop'],
                       uniparams['heritability'],
                       uniparams['meta_pop_sample_sizes'],
                       uniparams['number_of_replicates'])

ind_names_for_gwas = {i: {} for i in range(uniparams[
    'number_of_replicates'])}

founders = uniparams['founders']
replicated_nam = sim.Simulator(nam, rep=2, stealPops=False)
pop = replicated_nam.extract(0)

assert pop.popSize() == 26, "Population is too large."

s.generate_f_one(pop, recombination_rates, founders, 500)

In [ ]:
breeding_parameters = dict(offspring_per_founder_pair=uniparams['operating_population_size']/(len(founders)))

In [ ]:
breeding_parameters

In [1]:
expected_f_one_mother_ids = [1.0 for i in range(500)] + [3.0 for i in range(500)] \
+ [5.0 for i in range(500)] + [7.0 for i in range(500)]
expected_f_one_father_ids = [2.0 for i in range(500)] + [4.0 for i in range(500)] \
+ [6.0 for i in range(500)] + [8.0 for i in range(500)]

SyntaxError: unexpected character after line continuation character (<ipython-input-1-09977ce9a988>, line 2)

In [ ]:
observed_f_one_mother_ids = list(pop.indInfo('mother_id'))
observed_f_one_father_ids = list(pop.indInfo('father_id'))

In [ ]:
breeding_parameters['expected_f_one_mother_ids'] = expected_f_one_mother_ids
breeding_parameters['expected_f_one_father_ids'] = expected_f_one_father_ids

breeding_parameters['observed_f_one_mother_ids'] = observed_f_one_mother_ids
breeding_parameters['observed_f_one_father_ids'] = observed_f_one_father_ids

breeding_parameters['number_of_matches_f_one_mother_ids'] = sum(np.equal(expected_f_one_mother_ids, observed_f_one_mother_ids))
breeding_parameters['number_of_matches_f_one_father_ids'] = sum(np.equal(expected_f_one_father_ids, observed_f_one_father_ids))

In [ ]:
breeding_parameters['number_of_matches_f_one_father_ids']

In [ ]:
breeding_parameters['number_of_matches_f_one_mother_ids']

### Split the Offspring for Convienience

In [ ]:
pop.splitSubPop(0, [500] * 4)

In [ ]:
pop.setSubPopName('first', 0)
pop.setSubPopName('second', 1)
pop.setSubPopName('third', 2)
pop.setSubPopName('fourth', 3)

In [ ]:
first_sp_mothers = [random.choice(pop.indInfo('ind_id', 0)) for i in range(1000)]

second_sp_fathers = [random.choice(pop.indInfo('ind_id', 1)) for i in range(1000)]

third_sp_mothers = [random.choice(pop.indInfo('ind_id', 2)) for i in range(1000)]

fourth_sp_fathers = [random.choice(pop.indInfo('ind_id', 3)) for i in range(1000)]

### Merge Offspring After Choosing Parents

In [ ]:
pop.mergeSubPops()

In [ ]:
pop.subPopSizes()

In [ ]:
expected_f_two_mother_ids = first_sp_mothers + third_sp_mothers
expected_f_two_father_ids = second_sp_fathers + fourth_sp_fathers

In [ ]:
breeding_parameters['expected_f_two_mother_ids'] = expected_f_two_mother_ids
breeding_parameters['expected_f_two_father_ids'] = expected_f_two_father_ids

import importlib as imp
imp.reload(breed)

In [ ]:
second_order_pc = breed.SecondOrderPairIDChooser(expected_f_two_mother_ids, expected_f_two_father_ids)

In [ ]:
len(expected_f_two_father_ids)

In [ ]:
second_order_pc = breed.SecondOrderPairIDChooser(expected_f_two_mother_ids, expected_f_two_father_ids)

In [ ]:
pop.mergeSubPops()

In [ ]:
pop.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(second_order_pc.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.ParentsTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recombination_rates)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

In [ ]:
observed_f_two_mother_ids = list(pop.indInfo('mother_id'))
observed_f_two_father_ids = list(pop.indInfo('father_id'))

In [ ]:
breeding_parameters['expected_f_two_mother_ids'] = expected_f_two_mother_ids
breeding_parameters['expected_f_two_father_ids'] = expected_f_two_father_ids

breeding_parameters['observed_f_two_mother_ids'] = observed_f_two_mother_ids
breeding_parameters['observed_f_two_father_ids'] = observed_f_two_father_ids

breeding_parameters['number_of_matches_f_two_mother_ids'] = sum(np.equal(expected_f_two_mother_ids, observed_f_two_mother_ids))
breeding_parameters['number_of_matches_f_two_father_ids'] = sum(np.equal(expected_f_two_father_ids, observed_f_two_father_ids))

In [ ]:
breeding_parameters['number_of_matches_f_two_father_ids']

In [ ]:
breeding_parameters['number_of_matches_f_two_mother_ids']

## Determining Rounds of Mating Required for Convergence

Given *n* founders to start with:

If *n* is a power of two, *m*, then there are *m* rounds of mating.

The number of rounds of mating is the largest power of 2 which can
be subtracted from *n* + 1.

**Example**
```python

    founders = [1, 2, 3, 4, 5, 6, 7, 8]
    first_round = [(1, 2), (3, 4), (5, 6), (7, 8)]
    second_round = [(1/2, 3/4), (5/6, 7/8)]
    third_round = [(1/2 // 3/4, 5/6 // 7/8)]
    final_result = 1/2 // 3/4 /// 5/6 // 7/8
``` 

# Final Round of Mating

In [ ]:
pop.splitSubPop(0, sizes=[1000] * 2)

In [ ]:
final_first_sp_mothers = [random.choice(pop.indInfo('ind_id', 0)) for i in range(2000)]

final_second_sp_fathers = [random.choice(pop.indInfo('ind_id', 1)) for i in range(2000)]

In [ ]:
expected_f_final_mother_ids = final_first_sp_mothers
expected_f_final_father_ids = final_second_sp_fathers

In [ ]:
third_order_pc = breed.SecondOrderPairIDChooser(expected_f_final_mother_ids, expected_f_final_father_ids)

In [ ]:
pop.numSubPop()

In [ ]:
pop.mergeSubPops()

In [ ]:
pop.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(third_order_pc.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.ParentsTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recombination_rates)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

In [ ]:
observed_f_final_mother_ids = list(pop.indInfo('mother_id'))
observed_f_final_father_ids = list(pop.indInfo('father_id'))

In [ ]:
breeding_parameters['expected_f_two_mother_ids'] = expected_f_final_mother_ids
breeding_parameters['expected_f_two_father_ids'] = expected_f_final_father_ids

breeding_parameters['observed_f_two_mother_ids'] = observed_f_final_mother_ids
breeding_parameters['observed_f_two_father_ids'] = observed_f_final_father_ids

breeding_parameters['number_of_matches_f_final_mother_ids'] = sum(np.equal(expected_f_final_mother_ids, 
                                                                         observed_f_final_mother_ids))
breeding_parameters['number_of_matches_f_final_father_ids'] = sum(np.equal(expected_f_final_father_ids, 
                                                                         observed_f_final_father_ids))

In [ ]:
breeding_parameters['number_of_matches_f_final_mother_ids']

In [ ]:
breeding_parameters['number_of_matches_f_final_father_ids']

In [ ]:
pop.save("standard_magic.pop")

standard_magic_history = shelve.open("standard_magic_history")
for k, v in breeding_parameters.items():
    standard_magic_history[k] = v
standard_magic_history.close()